In [1]:
# Run our imports
import pandas as pd
import numpy as np
import yfinance as yf
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import re
import os


# INPUTS

In [2]:
# select Ticker
ticker = 'TSLA'

# Due to page load times, for now, for history we need to preload the page manually 
# and then download the html page to run against
url = 'file:///Users/danielgurzi/Desktop/Tesla,%20Inc.%20(TSLA)%20Latest%20Stock%20News%20_%20Seeking%20Alpha.html'

In [4]:
# spin up Selenium driver to get elements from the local html page
driver = webdriver.Chrome('./chromedriver')
driver.get(url)

links = driver.find_elements_by_class_name("_1-r1S")

In [5]:
# start dataframe to hold link information
df = pd.DataFrame()
df['date'] = [i.find_element_by_class_name("VbEwc").text for i in links]
df['name'] = [i.find_element_by_tag_name("h3").text for i in links]
df['href'] = [i.find_elements_by_tag_name('a')[0].get_property('href') for i in links]
# df['comments_qty'] = [i.find_element_by_class_name('_3zL6Q').text for i in links]

In [6]:
# this will end the browser session
driver.close()


In [7]:
# save info to dataframe
df.to_csv(f'./data/{ticker}_news_history.csv')


In [8]:
df

,date,name,href
0,"Thu, Jul. 02",Musk tweaks SEC again with Tesla shares cruising,https://seekingalpha.com/news/3588459-musk-twe...
1,"Thu, Jul. 02",Tesla revamps plan for Giga Berlin,https://seekingalpha.com/news/3588423-tesla-re...
2,"Thu, Jul. 02",Tesla Q2 deliveries top 90K vs. 83K estimate; ...,https://seekingalpha.com/news/3588220-tesla-q2...
3,"Wed, Jul. 01",Some clean energy ETFs shine in brutal first h...,https://seekingalpha.com/news/3587797-clean-en...
4,"Wed, Jul. 01",Tesla rallies more to become the most valuable...,https://seekingalpha.com/news/3587760-tesla-ra...
...,...,...,...
275,"Thu, Jan. 02",Canaccord Genuity fires off Street-high PT on ...,https://seekingalpha.com/news/3529084-canaccor...
276,"Thu, Jan. 02",First customers to receive China-made Model 3s,https://seekingalpha.com/news/3529033-first-cu...
277,"Wed, Jan. 01",Tesla faces lawsuit claiming racism,https://seekingalpha.com/news/3529018-tesla-fa...
278,"Tue, Dec. 31, 2019","Tesla Shanghai producing 1,000 cars a week",https://seekingalpha.com/news/3528861-tesla-sh...


In [9]:
# for the next part we'll need to clean the name field to get rid of slashes and punctuation
df['name'] = [i.replace('/', ' ') for i in df['name']]
df['name'] = [re.sub('[^A-Za-z0-9 $]+', '', i) for i in df['name']]


### Now we have a list of URLS from the seeking alpha ticker page

In [10]:
# now we open each html link saved in the dataframe and save the page_source content. We will most likely get errors, 
# so this saves the files we can save without errors. 
driver = webdriver.Chrome('./chromedriver')
for r,l in df.iterrows():
    driver.get(l[2])
    with open(f"./data/{ticker}_history/{l[1]}", "w") as f:
        f.write(driver.page_source)
driver.close()



In [13]:
# Function to go through the downloaded files and pull out the article content to save to a new dataframe
def qurom(df):
    #start a list of the articles
    art_list = []
    # grap the comments while I'm here
    comments = []
    # for loop to go through each of the links and grab the articles
    for filename in os.listdir(f'./data/{ticker}_history/'):
        driver.get(f'file:///Users/danielgurzi/GitHub/nlp_market_predict/data/TSLA_history/{filename}')
#         driver.implicitly_wait(10)
        try: 
            article = driver.find_element_by_id("a-cont")
        except:
            article = 'error'
        # The Article has comments, which for now I am going to take out. This code will separate the article into two sections with "Like this article" as the separator
        sep = 'Recommended for you:'
        try: 
            art_list.append(article.text.split(sep,1)[0])
        except:
            art_list.append('error')
        try: 
            comments.append(article.text.split(sep,1)[1])
        except:
            comments.append('error')
        print(len(art_list))
    df['article'] = art_list
    df['comments'] = comments
    return df

In [14]:
# open the browser
driver = webdriver.Chrome('./chromedriver')

df2 = qurom(df)
# df.to_csv(f'./data/{ticker}.csv', index = False)

# close the browser


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


ValueError: Length of values does not match length of index

In [30]:
driver.close()


InvalidSessionIdException: Message: invalid session id


In [247]:
# df3 = df2.drop('text', axis=1, inplace=True)
df3.to_csv(f'./data/{ticker}.csv', index = False)


KeyError: "['text'] not found in axis"

In [260]:
df3 = pd.read_csv(f'./data/{ticker}.csv')


In [262]:
df3['article'][0]

'Stocks To Watch: Fireworks For Nikola, Tesla And Micron\nJun. 27, 2020 9:00 AM ET|101 comments | Includes: AAXN, ACCD, ALO, ALXN, AMTI, AP, ARHH, ARNGF, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,\nSA Stocks To Watch\nLong/short equity, special situations, event-driven\n(55,141 followers)\nWelcome to Seeking Alpha\'s Stocks to Watch - a preview of key events scheduled for the next week. Follow this account and turn the e-mail alert on to receive this article in your inbox every Saturday morning. A podcast of Stocks to Watch is also available on Sundays on Seeking Alpha, Apple Podcasts, Stitcher and Spotify (click the highlighted links).\nFederal Reserve Chair Jerome Powell and Treasury Secretary Steven Mnuchin testify before Congress on June 30 in what\'s expected to be a broad overview of the economy and monetary policy. The Fed\'s John Willi

In [261]:
w2v_model = Word2Vec(sentences=df3['article'],
                     min_count=1,
                 window=5,
                 size=100,
                 workers=4)

# w2v_model.build_vocab(sentences)
w2v_model.train(tweets, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v_model.init_sims(replace=True)

,href,date,name,comments_qty,article,comments
0,https://seekingalpha.com/article/4355962-stock...,"Today, 9:00 AM","Stocks To Watch: Fireworks For Nikola, Tesla A...",92 Comments,"Stocks To Watch: Fireworks For Nikola, Tesla A...","\nDon't Buy Falling Financials, Buy These Stoc..."
1,https://seekingalpha.com/article/4355897-tesla...,"Yesterday, 2:05 PM",Tesla: Watching Q2 Coronavirus Deliveries,173 Comments,Earnings Analysis | Consumer\nTesla: Watching ...,\nThere Will Be No Second Wave\nLawrence Fulle...
2,https://seekingalpha.com/article/4355755-tesla...,"Thu, Jun. 25",Tesla: Ignoring Quality Issues Is No Longer An...,233 Comments,Tesla: Ignoring Quality Issues Is No Longer An...,\nTesla Makes An Expensive Decision (NASDAQ:TS...
3,https://seekingalpha.com/article/4355742-quest...,"Thu, Jun. 25","Questions Remain, But This May Now Be The Time...",115 Comments,"Long Ideas | Consumer \nQuestions Remain, But ...",\nIt Happened - The Second Wave Is Here\nJonat...
4,https://seekingalpha.com/article/4355640-tesla...,"Thu, Jun. 25",Tesla V2G Technology Could Dramatically Increa...,481 Comments,Long Ideas | Consumer \nTesla V2G Technology C...,\nHow To Retire With A Million In 10 Years\nFi...


In [122]:
driver = webdriver.Chrome('./chromedriver')
driver.get(url2)
driver.implicitly_wait(100)
article = driver.find_element_by_id("content-rail")
# assert "Python" in driver.title

# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
driver.close()

AssertionError: 

In [127]:
# The Article has comments, which for now I am going to take out. This code will separate the article into two sections with "Like this article" as the separator
sep = 'Recommended for you:'
article.text.split(sep,1)[1]

'\nDon\'t Buy Falling Financials, Buy These Stocks Instead\nLimelight Alpha Management Partners • Jun. 26, 2020 9:32 PM ET\n5 Reasons Bristol Myers Squibb Is An Exceptional Blue-Chip For Retirees (NYSE:BMY)\nBrad Thomas • Jun. 27, 2020 7:00 AM ET\nNikola: The S-1 Filing Is A Ticking Time Bomb For Longs (NASDAQ:NKLA)\nEdward Vranic, CFA • Jun. 24, 2020 8:36 AM ET\nWarren Buffett July Update: 46 Equities Show 9 To Buy And Hold Today\nFredrik Arnold • Jun. 25, 2020 4:59 PM ET\nComments 91\n• Sort by\n  Newest\n  Oldest\n  Most liked\nNewest\nAdd Your Comment\nPublish\namazapple\nComments14290 | + Follow\nThe universe of buzzy electric-vehicle stocks is about to get a new entrant: Hyliion, a Texas-based battery-powered heavy-duty truck company founded in 2015. It plans to merge with Tortoise Acquisition Corp. , a special-purpose acquisition company, by this fall.\nTortoise’s stock (ticker: SHLL) has already climbed nearly 70% since the announcement on June 19. Investors just can’t get enou

In [178]:
r = urllib.request.Request(url, headers={'User-Agent': 'Chrome/83.0.4103.116'})

In [179]:
html = urllib.request.urlopen(r)
time.sleep(10)

html = html.read()

In [174]:

# Establishing the connection to the web page:
response = requests.get(url)
print(response.status_code)

# Pull the HTML string out of requests and convert it to a Python string.
html = response.text


200


In [175]:
from requests_html import HTMLSession

session = HTMLSession()

r = session.get(url)

r.html.render()

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.

In [113]:
soup = BeautifulSoup(html,'lxml')

In [170]:
import urllib3
import time
data = urllib3.urlopen(url)
time.sleep(10)
data = data.read()

AttributeError: module 'urllib3' has no attribute 'urlopen'

In [15]:
results_list = []

articles = soup.find_all('article', {'class':'title'})

for element in articles:
    # start a dictionary to store this item's data
    result = {}
    
    # get the title and full link/url
    a_href = element.find('a')
    if a_href:
        result['title'] = a_href.text   # element text
        result['link'] = a_href['href'] # href link
        
    # get the url domain
    span = element.find('span', {'class':'comhead'})
    if span:
        result['url'] = span.text.strip()[1:-1]
        
    # only store "full" rows of data
    if len(result) == 3:
        results_list.append(result)
        
results_list[0]

IndexError: list index out of range